<a href="https://colab.research.google.com/github/shkailas/CementAnalysis/blob/main/PCA_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # helps with the math
import matplotlib.pyplot as plt # to plot error during training
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
%tensorflow_version 1.x
import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

import seaborn as sns

# %% Import Keras objects for Deep Learning

from keras.models  import Sequential, K
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
import sklearn

from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})



# %% Load in the data set F28 to Train Model
# Import the data using the file path

#filepath = '/content/drive/My Drive/Fish_Length.csv'

#filepath = '/content/drive/My Drive/Copy of _F28 Combo.xlsx'
filepath = "/content/_F28 Adj 1 extra.xlsx"
#saveFileName = "_Clinker MultiAge MultiMix Adj 1 trial 1.xls"
names=['','H','M','Al','Ca','Fe','Si']

df = pd.read_excel(filepath, header = 0)
numinput = int(46) # Number of input parameters

#df = pd.read_excel (r'C:\Users\pinka\Documents\Doctoral Thesis\Data Analysis\Machine Learning\_F28 Combo.xlsx',header=0) #Import .xlsx file
print('\n')
print(df.shape)
df.sample(5)

# %% Data Variables
X = df.iloc[:, 18:18+numinput].values # All rows, Third to 66th Column (IAl, ICa, IFe, ISi)
Y = df.iloc[:,1:2].values  # All rows, 2nd column (M)

print('\n')
print('Sample of X Data \n')
print(X[0:9]) # Confirm data was sucessfully assigned
print('\n')
print('Sample of Y Data \n')
print(Y[0:9]) # Confirm data was sucessfully assigned

# Split the data to Train, and Test (75%, 25%)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y,test_size=0.25, random_state=452452246)



(1181, 55)


Sample of X Data 

[[ 13 240  24 144  21 191  45  93  16 182  25 108  22 161  39  97   9 226
   45 121  15 239  23 144  12 226  78 121  16 224  40 139  22 230  39 164
   30]
 [ 16 224  40 139   9 226  45 121  13 240  24 144  15 239  23 144  12 226
   78 121  22 230  39 164  18 226  56 117  13 233  39 169  23 232  36 135
   30]
 [ 13 233  39 169  12 226  78 121  16 224  40 139  22 230  39 164  18 226
   56 117  23 232  36 135  35 229  89 114  14 238  46 167  17 230  38 151
   30]
 [ 14 238  46 167  18 226  56 117  13 233  39 169  23 232  36 135  35 229
   89 114  17 230  38 151  33 169  66  84  19 201  59  89  70 162 102  94
   30]
 [ 29 161  32  81  33 169  66  84  19 201  59  89  70 162 102  94  29 167
   23  83  45 113  48 156  34 164  72  58  68 171  65  57  19 146  75  69
   30]
 [ 68 171  65  57  29 167  23  83  29 161  32  81  45 113  48 156  34 164
   72  58  19 146  75  69  24 173  43  70  34 178  41  94  33 120  52 104
   30]
 [ 34 178  41  94  34 164  72  58  6

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
num = 20
pca = PCA(n_components=num)
pca.fit(X)
xp_var = pca.explained_variance_ratio_
print(np.cumsum(xp_var)[-1])
print(xp_var)
print(pca.components_[0])

In [ ]:
x_reduced = pca.fit_transform(X_train)
print(x_reduced)
x_reduced[1].size

[[-136.30175074  114.21850225   25.49337994 ...  -35.94254519
    32.69634056  -24.56192787]
 [-189.35250043   95.91303186   51.86625178 ...   26.95206224
    13.02846192  -28.98563452]
 [-269.07346738   22.21407278  -63.53300313 ...  -20.01187965
     0.44060843  -30.60854515]
 ...
 [ -60.25879449   80.83350762 -113.61469615 ...   31.92931955
   -42.26110424  -11.72317231]
 [  32.95424103    8.44798642  176.97363523 ...   13.04419017
    54.91458303   35.93500512]
 [  38.11244179   89.86353547   66.87518676 ...    8.54210803
    23.16880041   -9.63404442]]


20

In [ ]:
#beginning of randomizedSearchCV code
from keras import optimizers
#RandomizedSearchCV with NN#####################################################

#create a function to make a neural net

def create_model(nl1=1, nl2=1, nn1=1000, nn2=500, lr=0.01, decay=0., l1=0.01,
                 l2=0.01, act1 = 'relu', dropout=0, input_shape=num,
                output_shape=1, optim = 1):
    #'''This is a model generating function so that we can search over neural net 
    #parameters and architecture'''
        
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    # for the firt layer we need to specify the input dimensions
    first=True
    
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act1, 
                            kernel_regularizer=reg))
            model.add(BatchNormalization())
            first=False
        else: 
            model.add(Dense(nn1, activation=act1, kernel_regularizer=reg))
            model.add(BatchNormalization())
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act1, 
                            kernel_regularizer=reg))
            model.add(BatchNormalization())
            first=False
        else: 
            model.add(Dense(nn2, activation=act1, kernel_regularizer=reg))
            model.add(BatchNormalization())
        if dropout!=0:
            model.add(Dropout(dropout))
            
            
    model.add(Dense(output_shape, activation='linear'))
    if optim == 1:
      model.compile(loss='mean_absolute_error', optimizer=opt)
    else:
      model.compile(loss='mean_absolute_error', optimizer=sgd)  

    return model


In [ ]:
#List parameters to test in randomSearchCV
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# model class to use in the scikit random search CV 
model3 = KerasRegressor(build_fn=create_model, epochs=15, batch_size=20, 
                        verbose=1)
  
# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# activation
activation=['relu', 'swish']

# numbers of layers
nl1 = [1,2]
nl2 = [1,2]


# neurons in each layer
nn1=[5,10,15,20,25]
nn2=[8,17,25]


# dropout and regularisation
dropout = [0, 0.1, 0.2, 0.3]
l1 = [0, 0.01, 0.003, 0.001,0.0001]
l2 = [0, 0.01, 0.003, 0.001,0.0001]

#optimizer Choices(1=adam, 2 = sgd)
optims = [1,2]

# dictionary summary
param_grid = dict(
                    nl1=nl1, nl2=nl2, nn1=nn1, nn2=nn2, 
                    act1=activation, l1=l1,
                    l2=l2, lr=lr, decay=decay, dropout=dropout,
                    optim =  optims
                    #input_shape=[X.shape[1]], output_shape = [y.shape[1]],
                 )
grid = RandomizedSearchCV(estimator=model3, param_distributions=param_grid,
                          verbose=20,  n_iter=40, n_jobs=1,cv = 3)

In [ ]:
grid_result = grid.fit(x_reduced, Y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] optim=1, nn2=25, nn1=25, nl2=1, nl1=2, lr=0.01, l2=0.0001, l1=0.001, dropout=0.2, decay=0, act1=swish 



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.






Epoch 1/15





590/590 [==============================] - 10s 16ms/step - loss: 43.8514
Epoch 2/15
590/590 [==============================] - 0s 547us/step - loss: 37.4087
Epoch 3/15
590/590 [==============================] - 0s 559us/step - loss: 22.5733
Epoch 4/15
590/590 [==============================] - 0s 600us/step - loss: 13.2156
Epoch 5/15
590/590 [==============================] - 0s 575us/step - loss: 12.6430
Epoch 6/15
590/590 [==============================] - 0s 587us/step - loss: 12.6539
Epoch 7/15
590/590 [==============================] - 0s 546us/step - loss: 12.6489
Epoch 8/15
590/590 [==============================] - 0s 580us/step - loss: 12.1963
Epoch 9/15
590/590 [==============================] - 0s 599us/step - loss: 12.3493
Epoch 10/15
590/590 [==============================] - 0s 560us/step - loss: 12.0017
Epoch 11/15
590/590 [==============================] - 0s 565us/step - loss: 11.4819
Epoch 12/15
590/590 [==============================] - 0s 556us/s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.1s remaining:    0.0s


Epoch 1/15
590/590 [==============================] - 1s 2ms/step - loss: 44.3442
Epoch 2/15
590/590 [==============================] - 0s 552us/step - loss: 37.4832
Epoch 3/15
590/590 [==============================] - 0s 580us/step - loss: 23.0354
Epoch 4/15
590/590 [==============================] - 0s 569us/step - loss: 14.5973
Epoch 5/15
590/590 [==============================] - 0s 555us/step - loss: 14.1332
Epoch 6/15
590/590 [==============================] - 0s 551us/step - loss: 13.1990
Epoch 7/15
590/590 [==============================] - 0s 589us/step - loss: 12.8139
Epoch 8/15
590/590 [==============================] - 0s 539us/step - loss: 12.9946
Epoch 9/15
590/590 [==============================] - 0s 595us/step - loss: 12.4638
Epoch 10/15
590/590 [==============================] - 0s 603us/step - loss: 12.3307
Epoch 11/15
590/590 [==============================] - 0s 600us/step - loss: 12.2462
Epoch 12/15
590/590 [==============================] - 0s 559us/step - loss:

...

Epoch 1/15
590/590 [==============================] - 4s 7ms/step - loss: 45.0075
Epoch 2/15
590/590 [==============================] - 0s 461us/step - loss: 44.4215
Epoch 3/15
590/590 [==============================] - 0s 469us/step - loss: 43.6745
Epoch 4/15
590/590 [==============================] - 0s 467us/step - loss: 42.6674
Epoch 5/15
590/590 [==============================] - 0s 489us/step - loss: 41.1751
Epoch 6/15
590/590 [==============================] - 0s 449us/step - loss: 38.8190
Epoch 7/15
590/590 [==============================] - 0s 480us/step - loss: 34.9474
Epoch 8/15
590/590 [==============================] - 0s 477us/step - loss: 28.5335
Epoch 9/15
590/590 [==============================] - 0s 515us/step - loss: 19.1813
Epoch 10/15
590/590 [==============================] - 0s 538us/step - loss: 13.0295
Epoch 11/15
590/590 [==============================] - 0s 455us/step - loss: 12.4517
Epoch 12/15
590/590 [==============================] - 0s 496us/step - loss:

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:  3.0min remaining:    0.0s


Epoch 1/15
590/590 [==============================] - 4s 7ms/step - loss: 45.7124
Epoch 2/15
590/590 [==============================] - 0s 476us/step - loss: 44.9166
Epoch 3/15
590/590 [==============================] - 0s 502us/step - loss: 43.8662
Epoch 4/15
590/590 [==============================] - 0s 477us/step - loss: 42.3783
Epoch 5/15
590/590 [==============================] - 0s 477us/step - loss: 40.0730
Epoch 6/15
590/590 [==============================] - 0s 538us/step - loss: 36.3123
Epoch 7/15
590/590 [==============================] - 0s 527us/step - loss: 30.0229
Epoch 8/15
590/590 [==============================] - 0s 525us/step - loss: 20.7386
Epoch 9/15
590/590 [==============================] - 0s 489us/step - loss: 13.9456
Epoch 10/15
590/590 [==============================] - 0s 468us/step - loss: 13.0562
Epoch 11/15
590/590 [==============================] - 0s 504us/step - loss: 12.8997
Epoch 12/15
590/590 [==============================] - 0s 497us/step - loss:

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 52.0min finished


Epoch 1/15
885/885 [==============================] - 27s 31ms/step - loss: 43.7834
Epoch 2/15
885/885 [==============================] - 1s 614us/step - loss: 32.4193
Epoch 3/15
885/885 [==============================] - 1s 654us/step - loss: 15.5860
Epoch 4/15
885/885 [==============================] - 1s 616us/step - loss: 13.0939
Epoch 5/15
885/885 [==============================] - 1s 637us/step - loss: 12.8556
Epoch 6/15
885/885 [==============================] - 1s 653us/step - loss: 12.5256
Epoch 7/15
885/885 [==============================] - 1s 649us/step - loss: 12.1773
Epoch 8/15
885/885 [==============================] - 1s 616us/step - loss: 12.1011
Epoch 9/15
885/885 [==============================] - 1s 647us/step - loss: 11.9773
Epoch 10/15
885/885 [==============================] - 1s 619us/step - loss: 12.0443
Epoch 11/15
885/885 [==============================] - 1s 624us/step - loss: 12.0206
Epoch 12/15
885/885 [==============================] - 1s 632us/step - los

In [ ]:
grid_result.best_estimator_
grid_result.best_params_
#grid_result.best_score_


{'act1': 'relu',
 'decay': 0,
 'dropout': 0.2,
 'l1': 0,
 'l2': 0,
 'lr': 0.01,
 'nl1': 1,
 'nl2': 1,
 'nn1': 20,
 'nn2': 17,
 'optim': 1}

In [ ]:
from keras.callbacks import EarlyStopping
model = grid_result.best_estimator_
#earlyStopping for val data
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, 
                 mode='auto', baseline=None, restore_best_weights=True)
#model.compile(loss = "mean_squared_error", optimizer = "adam")
model.fit(x_reduced, Y_train, epochs = 100, validation_split = 0.2, 
          callbacks=[es])
x_test_transform = pca.transform(X_test)
model.score(x_test_transform,Y_test)

Train on 708 samples, validate on 177 samples
Epoch 1/100
708/708 [==============================] - 32s 45ms/step - loss: 44.6773 - val_loss: 39.9443
Epoch 2/100
708/708 [==============================] - 0s 666us/step - loss: 37.6999 - val_loss: 28.3116
Epoch 3/100
708/708 [==============================] - 1s 708us/step - loss: 22.8860 - val_loss: 11.5520
Epoch 4/100
708/708 [==============================] - 0s 653us/step - loss: 14.5363 - val_loss: 11.0230
Epoch 5/100
708/708 [==============================] - 0s 692us/step - loss: 13.5780 - val_loss: 10.8868
Epoch 6/100
708/708 [==============================] - 0s 696us/step - loss: 13.1406 - val_loss: 10.5136
Epoch 7/100
708/708 [==============================] - 0s 691us/step - loss: 12.4406 - val_loss: 9.8892
Epoch 8/100
708/708 [==============================] - 1s 709us/step - loss: 12.6266 - val_loss: 9.9013
Epoch 9/100
708/708 [==============================] - 0s 673us/step - loss: 12.4178 - val_loss: 10.2240
Epoch 10/10

In [ ]:
print(np.mean(Y))
print(np.std(Y))

45.30206604572397
23.479383628817917
